TFX version 0.21.2

TF version 2.1.0

In [26]:
!pip install tensorflow==2.2.0rc0

     |████████████████████████████████| 175.2MB 4.7MB/s eta 0:00:01    |█████▊                          | 31.4MB 6.3MB/s eta 0:00:23     |███████████▉                    | 65.0MB 4.3MB/s eta 0:00:26     |████████████▌                   | 68.5MB 1.1MB/s eta 0:01:37     |█████████████                   | 71.7MB 1.1MB/s eta 0:01:34     |█████████████▋                  | 74.6MB 7.0MB/s eta 0:00:15     |████████████████▋               | 90.9MB 3.9MB/s eta 0:00:22     |█████████████████▊              | 97.1MB 5.5MB/s eta 0:00:15     |██████████████████              | 99.1MB 5.5MB/s eta 0:00:14     |█████████████████████▉          | 119.4MB 6.3MB/s eta 0:00:09     |███████████████████████▏        | 127.0MB 5.3MB/s eta 0:00:10     |██████████████████████████████  | 164.0MB 3.3MB/s eta 0:00:04     |██████████████████████████████▍ | 166.6MB 2.3MB/s eta 0:00:04
ERROR: tensorflow-transform 0.21.2 has requirement tensorflow<2.2,>=1.15, but you'll have tensorflow 2.2.0rc0 which is incompatible.
ERRO

In [1]:
import os
import sys

import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from tfx.utils.dsl_utils import external_input
from tfx.components import CsvExampleGen

In [2]:
!pip show tfx

Name: tfx
Version: 0.21.2
Summary: TensorFlow Extended (TFX) is a TensorFlow-based general-purpose machine learning platform implemented at Google
Home-page: https://www.tensorflow.org/tfx
Author: Google LLC
Author-email: tensorflow-extended-dev@googlegroups.com
License: Apache 2.0
Location: /Users/i854694/.virtualenvs/pipelines/lib/python3.7/site-packages
Requires: docker, protobuf, tensorflow-data-validation, tensorflow-model-analysis, google-api-python-client, tensorflow-serving-api, pyarrow, tfx-bsl, absl-py, apache-beam, click, ml-metadata, grpcio, six, tensorflow, tensorflow-transform, jinja2, pyyaml
Required-by: 


In [3]:
# add project to the python path
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [4]:
context = InteractiveContext(pipeline_root='../tfx/10Mar')

In [5]:
base_dir = os.getcwd()
data_dir = "../data/6Mar/"
examples = external_input(os.path.join(base_dir, data_dir))
example_gen = CsvExampleGen(input=examples)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 13
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ../tfx/10Mar/CsvExampleGen/examples/13, id: 15)]
        ))

In [6]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

/Users/i854694/.virtualenvs/pipelines/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 14
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: ../tfx/10Mar/StatisticsGen/statistics/14, id: 16)]
        ))

In [7]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [8]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 15
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: ../tfx/10Mar/SchemaGen/schema/15, id: 17)]
        ))

In [9]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',BYTES,required,,-
'company_response',STRING,required,,'company_response'
'consumer_complaint_narrative',BYTES,required,,-
'consumer_disputed',STRING,required,,'consumer_disputed'
'issue',STRING,required,,'issue'
'product',STRING,required,,'product'
'state',STRING,optional,single,'state'
'sub_issue',STRING,optional,single,'sub_issue'
'sub_product',STRING,optional,single,'sub_product'


,Values
Domain,
'company_response',"'Closed', 'Closed with explanation', 'Closed with monetary relief', 'Closed with non-monetary relief', 'Untimely response'"
'consumer_disputed',"'No', 'Yes'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Excessive fees', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender damaged or destroyed vehicle', 'Lender repossessed or sold the vehicle', 'Lender sold the property', 'Loan modification,collection,foreclosure', 'Loan servicing, payments, escrow account', 'Lost or stolen check', 'Lost or stolen money order', 'Making/receiving payments, sending money', 'Managing the line of credit', 'Managing the loan or lease', 'Managing, opening, or closing account', 'Money was not available when promised', 'Other', 'Other fee', 'Other service issues', 'Other transaction issues', 'Overdraft, savings or rewards features', 'Overlimit fee', 'Payment to acct not credited', 'Payoff process', 'Privacy', 'Problems caused by my funds being low', 'Problems when you are unable to pay', 'Received a loan I didn\'t apply for', 'Rewards', 'Sale of account', 'Settlement process and costs', 'Shopping for a line of credit', 'Shopping for a loan or lease', 'Taking out the loan or lease', 'Taking/threatening an illegal action', 'Transaction issue', 'Unable to get credit report/credit score', 'Unauthorized transactions/trans. issues', 'Unexpected/Other fees', 'Unsolicited issuance of credit card', 'Using a debit or ATM card', 'Wrong amount charged or received'"
'product',"'Bank account or service', 'Consumer Loan', 'Credit card', 'Credit reporting', 'Debt collection', 'Money transfers', 'Mortgage', 'Other financial service', 'Payday loan', 'Prepaid card', 'Student loan'"
'state',"'AA', 'AE', 'AK', 'AL', 'AP', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'FM', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'"
'sub_issue',"'Account status', 'Account terms', 'Account terms and changes', 'Applied for loan/did not receive money', 'Attempted to collect wrong amount', 'Attempted to/Collected exempt funds', 'Billing dispute', 'Called after sent written cease of comm', 'Called outside of 8am-9pm', 'Can\'t contact lender', 'Can\'t decrease my monthly payments', 'Can\'t get flexible payment options', 'Can\'t qualify for a loan', 'Can\'t stop charges to bank account', 'Can\'t temporarily postpone payments', 'Charged bank acct wro

In [10]:
from tfx.components import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'issue',Unexpected string values,"Examples contain values missing from the schema: Disclosures (<1%), Incorrect exchange rate (<1%)."


In [11]:
transform_file = os.path.join(base_dir, '../components/transform.py')

In [12]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=transform_file)
context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
Use ref() instead.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: ../tfx/10Mar/Transform/transform_graph/17/.temp_path/tftransform_tmp/fc4bb517470d44398c59636d07b3737f/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' obj

ExecutionResult(
    component_id: Transform
    execution_id: 17
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: ../tfx/10Mar/Transform/transform_graph/17, id: 19)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ../tfx/10Mar/Transform/transformed_examples/17, id: 20)]
        ))

In [13]:
import pprint
import tensorflow_data_validation as tfdv

tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = transform.outputs['transformed_examples'].get()[0].uri + '/train'

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a TFRecordDataset to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and print them
for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "company_response_xf"
    value {
      float_list {
        value: 0.0
        value: 1.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
      }
    }
  }
  feature {
    key: "consumer_complaint_narrative_xf"
    value {
      bytes_list {
        value: "my mortgage servicing provider   xxxx   transferred my mortgage to another service provider   suntrust    this is a standard procedure and i went through the process several times before  however  this time i was not informed about the transfer from either company  because xxxx had a wrong address on file i did not receive their notifications  additionally  since the new service provider   suntrust   relies on the address from the previous service provider  i did not receive any communication from them either    i found out about the transfer a few months later via a coincidence  i checked my credit report and noticed that my credit score dropped  apparently i had a mortga

In [14]:
trainer_file = os.path.join(base_dir, '../components/keras_trainer.py')

In [15]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor

TRAINING_STEPS = 2000
EVALUATION_STEPS = 100

trainer = Trainer(
    module_file=trainer_file,
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
consumer_complaint_narrative_xf [(None, 1)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_Reshape (TensorFlow [(None,)]            0           consumer_complaint_narrative_xf[0
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 512)          256797824   tf_op_layer_Reshape[0][0]        
__________________________________________________________________________________________________
reshape (Reshape)               (None, 512)          0           keras_layer[0][0]                
______________________________________________________________________________________________

ExecutionResult(
    component_id: Trainer
    execution_id: 18
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: ../tfx/10Mar/Trainer/model/18, id: 21)]
        ))

In [16]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri

print(os.listdir(model_artifact_dir))

['serving_model_dir', 'logs']


In [26]:
model_artifact_dir

'../tfx/10Mar/Trainer/model/18'

In [31]:
trainer = Trainer(
    module_file=trainer_file,
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

ExecutionResult(
    component_id: Trainer
    execution_id: 22
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: , id: 0)]
        ))

In [32]:
log_dir = os.path.join(model_artifact_dir, 'logs')

%load_ext tensorboard
%tensorboard --logdir {log_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 19505), started 0:00:18 ago. (Use '!kill 19505' to kill it.)

In [21]:
from tfx.components import ResolverNode
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing


model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))
context.run(model_resolver)

ExecutionResult(
    component_id: ResolverNode.latest_blessed_model_resolver
    execution_id: 20
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
        ))

In [24]:
eval_config=tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
    slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
    metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(class_name='BinaryAccuracy')])]

    )

In [25]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator)

	 [[{{node StatefulPartitionedCall/transform/transform/inputs/F_consumer_disputed}}]] [Op:__inference_signature_wrapper_172588]

Function call stack:
signature_wrapper
. Attempting to run batch through serially.


RuntimeError: tensorflow.python.framework.errors_impl.InvalidArgumentError:  You must feed a value for placeholder tensor 'StatefulPartitionedCall/transform/transform/inputs/F_consumer_disputed' with dtype string and shape [?,1]
	 [[{{node StatefulPartitionedCall/transform/transform/inputs/F_consumer_disputed}}]] [Op:__inference_signature_wrapper_172588]

Function call stack:
signature_wrapper [while running 'ExtractEvaluateAndWriteResults/ExtractAndEvaluate/ExtractPredictions/Predict']

# STOP HERE FOR NOW

In [17]:
# todo: transform labels https://github.com/tensorflow/model-analysis/issues/59

import tensorflow_model_analysis as tfma
from tfx.components import Evaluator

model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])]
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                  tfma.MetricConfig(class_name='BinaryAccuracy')])]
        
        ))
context.run(model_analyzer)

	 [[{{node StatefulPartitionedCall/transform/transform/inputs/F_consumer_disputed}}]] [Op:__inference_signature_wrapper_123849]

Function call stack:
signature_wrapper
. Attempting to run batch through serially.


RuntimeError: tensorflow.python.framework.errors_impl.InvalidArgumentError:  You must feed a value for placeholder tensor 'StatefulPartitionedCall/transform/transform/inputs/F_consumer_disputed' with dtype string and shape [?,1]
	 [[{{node StatefulPartitionedCall/transform/transform/inputs/F_consumer_disputed}}]] [Op:__inference_signature_wrapper_123849]

Function call stack:
signature_wrapper [while running 'ExtractEvaluateAndWriteResults/ExtractAndEvaluate/ExtractPredictions/Predict']

In [ ]:
#NB TFMA will not run in jupyterlab

In [ ]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = model_analyzer.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='product')

In [ ]:
from tfx.components.model_validator.component import ModelValidator

model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'])
context.run(model_validator)

In [ ]:
!mkdir -p ../tfx/serving_model_dir

In [ ]:

from tfx.components.pusher.component import Pusher
from tfx.proto import pusher_pb2

_serving_model_dir = "/tfx/serving_model_dir"

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)